In [1]:
import pandas as pd
import numpy as np

In [3]:
# 분석할 데이터 load -> auto-mpg.csv 파일 가져 옴
df = pd.read_csv('dataset/auto-mpg.csv',header=None)

df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name']

df.info()
df['horsepower'].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


0    130.0
1    165.0
2    150.0
3    150.0
4    140.0
Name: horsepower, dtype: object

In [4]:
# 데이터 전처리
# horsepower를 구간으로 나누어 분석, 데이터 타입 object -> float
# df['horsepower'] = df['horsepower'].astype('float') # '?'를 찾음
# '?'를 NaN으로 처리 후 행을 삭제
df['horsepower'].replace('?',np.nan,inplace=True)
df.dropna(subset=['horsepower'],axis=0,inplace=True)

In [6]:
df['horsepower'] = df['horsepower'].astype('float')

In [9]:
# horsepower를 구간으로 나누어 분석 -> 구간을 np.histogram
count, bin_value = np.histogram(df['horsepower'],bins=3)
print(count, bin_value)

# 구간의 이름을 부여
bin_names = ['저출력','보통출력','고출력']

# 분석하고자 하는 자료에 bin_names를 추가
df['hp_bin'] = pd.cut(x=df['horsepower'], # 처리할 데이터
                      bins=bin_value, # 경계값 리스트
                      labels = bin_names, # 구간의 이름 리스트
                      include_lowest='True')

[257 103  32] [ 46.         107.33333333 168.66666667 230.        ]


In [13]:
df[['horsepower','hp_bin']].head(20)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [34]:
# 분류형 자료를 더미 변수로 전환
h_dummy = pd.get_dummies(df['hp_bin'],prefix='hp')
h_dummy
pd.concat([df,h_dummy],axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           392 non-null    float64 
 1   cylinders     392 non-null    int64   
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   weight        392 non-null    float64 
 5   acceleration  392 non-null    float64 
 6   model year    392 non-null    int64   
 7   origin        392 non-null    int64   
 8   name          392 non-null    object  
 9   hp_bin        392 non-null    category
dtypes: category(1), float64(5), int64(3), object(1)
memory usage: 31.1+ KB


In [30]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.alive.unique()
alive_yes = pd.get_dummies(titanic['alive'],prefix='alive',drop_first=True)

In [35]:
df1 = pd.concat([titanic,alive_yes],axis=1)
df1[['alive','alive_yes']].head()

,alive,alive_yes
0,no,0
1,yes,1
2,yes,1
3,yes,1
4,no,0


In [41]:
# one_hot encoding으로 더미 변수 생성
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 생성
label_encoder = preprocessing.LabelEncoder() # label_encoder 생성
onehot_encoder = preprocessing.OneHotEncoder() # onehot_encoder 생성

# label_encoder 문자열 함수를 숫자형 범주로 변환
onehot_label = label_encoder.fit_transform(df['hp_bin'])
print(onehot_label.ndim, onehot_label.dtype,onehot_label.size)

# 2차원 행렬로 변환
onehot_reshape = onehot_label.reshape(len(onehot_label),1)
print(onehot_reshape.ndim,onehot_reshape.dtype,onehot_reshape.size)
onehot_reshape[:10,:]

# 희소 행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshape)
print(onehot_fitted)

1 int32 392
2 int32 392
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 2)	1.0
  (17, 2)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 2)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 1)	1.0
  (24, 2)	1.0
  :	:
  (367, 2)	1.0
  (368, 2)	1.0
  (369, 2)	1.0
  (370, 2)	1.0
  (371, 2)	1.0
  (372, 2)	1.0
  (373, 2)	1.0
  (374, 2)	1.0
  (375, 2)	1.0
  (376, 2)	1.0
  (377, 2)	1.0
  (378, 2)	1.0
  (379, 2)	1.0
  (380, 1)	1.0
  (381, 2)	1.0
  (382, 2)	1.0
  (383, 1)	1.0
  (384, 2)	1.0
  (385, 2)	1.0
  (386, 2)	1.0
  (387, 2)	1.0
  (388, 2)	1.0
  (389, 2)	1.0
  (390, 2)	1.0
  (391, 2)	1.0


In [57]:
# 시계열 데이터 처리
df = pd.read_csv('dataset/stock-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


In [58]:
df['new_Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB


In [59]:
# new_Date 인덱스 컬럼으로 설정, Date 컬럼을 삭제
df.set_index(['new_Date'],inplace=True)
df.head()

df.drop('Date',axis=1,inplace=True)
df.head()

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


In [62]:
# 날짜 데이터 분리 년- 월- 일
df = pd.read_csv('dataset/stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day

df

,Date,Close,Start,High,Low,Volume,new_Date,Year,Month,Day
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02,2018,7,2
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29,2018,6,29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28,2018,6,28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27,2018,6,27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26,2018,6,26
5,2018-06-25,11150,11400,11450,11000,55519,2018-06-25,2018,6,25
6,2018-06-22,11300,11250,11450,10750,134805,2018-06-22,2018,6,22
7,2018-06-21,11200,11350,11750,11200,133002,2018-06-21,2018,6,21
8,2018-06-20,11550,11200,11600,10900,308596,2018-06-20,2018,6,20
9,2018-06-19,11300,11850,11950,11300,180656,2018-06-19,2018,6,19


In [85]:
# titanic 데이터를 로드
df = sns.load_dataset('titanic')
df.info()
# NaN 데이터를 처리: age는 평균값으로, deck은 삭제
df['age'].fillna(df['age'].mean(axis=0),inplace=True)
df['age']

df.dropna(subset=['deck'],thresh=300,inplace=True)

df[['embarked','embark_town']].fillna(method='ffill',inplace=True)
df.info()

# 다른 컬럼은 이전 값으로 대체
# 평균 age보다 적은 age의 승객 중에서 여성과 생존자만 age, sex, alive, class 컬럼 추출
col_list = ['age','sex','alive','class']
mask1 = (df['age']<df['age'].mean()) & (df.sex=='female') & (df['alive']=='yes')
df_1 = df.loc[mask1,col_list]
df_1
# 가족의 수가 4,5명인 승객의 나이, 가족수, 생존여부를 추출
df.loc[df['sibsp'].isin([4,5]), ['age','sibsp','alive']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,age,sibsp,alive


In [91]:
df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']},
                    index=[0, 1, 2, 3])
 
df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']},
                    index=[2, 3, 4, 5])

result1 = pd.concat([df1,df2])
result1

# 기존의 인덱스를 무시하고 인덱스 재설정
result2 = pd.concat([df1,df2],ignore_index=True)
print(result2)

# 열 단위로 합성
result3 = pd.concat([df1,df2],axis=1) # 인덱스가 없으면 NaN
print(result3)

# 인덱스가 같은 행만 조인
result4 = pd.concat([df1,df2],axis=1,join='inner')
print(result4)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
4  a2  b2  c2   d2
5  a3  b3  c3   d3
6  a4  b4  c4   d4
7  a5  b5  c5   d5
     a    b    c    a    b    c    d
0   a0   b0   c0  NaN  NaN  NaN  NaN
1   a1   b1   c1  NaN  NaN  NaN  NaN
2   a2   b2   c2   a2   b2   c2   d2
3   a3   b3   c3   a3   b3   c3   d3
4  NaN  NaN  NaN   a4   b4   c4   d4
5  NaN  NaN  NaN   a5   b5   c5   d5
    a   b   c   a   b   c   d
2  a2  b2  c2  a2  b2  c2  d2
3  a3  b3  c3  a3  b3  c3  d3


In [97]:
sr1 = pd.Series(['e0','e1','e2','e3'],name='e')
sr2 = pd.Series(['f0','f1','f2'],name='f',index=[3,4,5])
sr3 = pd.Series(['g0','g1','g2','g3'],name='g')

# df1에 sr1을 추가
res = pd.concat([df1,sr1],axis=1)
res

,a,b,c,e
0,a0,b0,c0,e0
1,a1,b1,c1,e1
2,a2,b2,c2,e2
3,a3,b3,c3,e3


In [100]:
# df2에 sr2를 추가
res1 = pd.concat([df2,sr2],axis=1,sort=True)
res1

,a,b,c,d,f
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,f0
4,a4,b4,c4,d4,f1
5,a5,b5,c5,d5,f2


In [102]:
# sr1과 sr3를 결합
res2 = pd.concat([sr1,sr3],axis=1)
res2

,e,g
0,e0,g0
1,e1,g1
2,e2,g2
3,e3,g3


In [106]:
# 데이터프레임 병합: merge()
df1 = pd.read_excel('dataset/stock_price.xlsx')
df2 = pd.read_excel('dataset/stock_valuation.xlsx')

print(df1)
print()
print(df2)
print()

merge_inner = pd.merge(df1,df2) # inner join, 처음 컬럼 기준
print(merge_inner)

       id stock_name          value   price
0  128940       한미약품   59385.666667  421000
1  130960     CJ E&M   58540.666667   98900
2  138250      엔에스쇼핑   14558.666667   13200
3  139480        이마트  239230.833333  254500
4  142280     녹십자엠에스     468.833333   10200
5  145990        삼양사   82750.000000   82000
6  185750        종근당   40293.666667  100500
7  192400      쿠쿠홀딩스  179204.666667  177500
8  199800         툴젠   -2514.333333  115400
9  204210     모두투어리츠    3093.333333    3475

       id       name           eps     bps        per       pbr
0  130960     CJ E&M   6301.333333   54068  15.695091  1.829178
1  136480         하림    274.166667    3551  11.489362  0.887074
2  138040    메리츠금융지주   2122.333333   14894   6.313806  0.899691
3  139480        이마트  18268.166667  295780  13.931338  0.860437
4  145990        삼양사   5741.000000  108090  14.283226  0.758627
5  161390      한국타이어   5648.500000   51341   7.453306  0.820007
6  181710  NHN엔터테인먼트   2110.166667   78434  30.755864  0.827447
7  

In [107]:
merge_outer = pd.merge(df1,df2,how='outer',on='id')
merge_outer

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400.0,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359
